# Six-bar mechanism balancing

###### Luis Eduardo Robles Jiménez

With this work, I pretend to find out the behavior of the function with different tools, in order to understand what are the most important variables, find patterns, identify overlapping zones of minima between the functions we're optimizing, graphing data, etc.

In [4]:
from BetaShF import *
from BetaShM import *
import numpy as np
import scipy as sp

assert ShF([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) == 1.0
assert ShM([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) == 1.0

#### Constraints

$$ -0.16m \leq x_{cn}, y_{cn} \leq 0.16m $$

$$ 0.005m \leq t_{cn} \leq 0.04m $$

#### Objective function

In [5]:
def objective(s, ShF, ShM):
    return 0.5*ShF(s) + 0.5*ShM(s)

## Mathematical analysis with sympy

## Sample generation

In [6]:
nSamples, nVariables = 250, 15
samples = np.zeros((nSamples, nVariables))
bounds = np.zeros((nVariables, 2))
fitness = np.zeros((nSamples))
"""
for i in range(nVariables):
    if (i + 1) % 3 == 0: samples[:, i] = np.random.uniform(0.005, 0.04, (nSamples))
    else: samples[:, i] = np.random.uniform(-0.16, 0.16, (nSamples))
"""
for v in range(nVariables):
    bounds[v] = [0.005, 0.04] if (v + 1) % 3 == 0 else [-0.16, 0.16]
worst = 0.0
for s in range(nSamples):
    print("\rIteration:", s, "\tWorst fitness:", worst, end = "")
    r = sp.optimize.differential_evolution(objective, bounds, args = (ShF, ShM), maxiter = 10)
    samples[s] = r.x
    fitness[s] = r.fun
    worst = np.max([worst, fitness[s]])

Iteration: 249 	Worst fitness: 0.7156200975691219

In [7]:
import time
now = str(int(time.time()))
np.savetxt(now + "Population.txt", samples)
np.savetxt(now + "Fitness.txt", fitness)

## Dimensionality reduction

### Feature extraction

### Feature selection

## Data visualization